<a href="https://colab.research.google.com/github/GrupoTuring/Fake-Citation/blob/main/fake_citations_badmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import torch
import nltk
from nltk import word_tokenize
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


nltk.download('punkt')

In [3]:
dataset = pd.read_csv("/content/fakequote.csv", na_values=['\r\n', ' '])
dataset.drop(columns=['title', 'url'], inplace=True)
dataset.dropna(inplace=True)
print("Número de linhas ", dataset.shape[0])
dataset.head()

Número de linhas  1336


,citation,category
1,Não se justifica um bilionário aporte financei...,CONTRADITÓRIO
2,A questão de vacinação ou não é uma lei de 75 ...,EXAGERADO
3,"Inclusive tem um vídeo, do senhor João Doria, ...",VERDADEIRO
4,Hoje em dia pelo menos metade da população diz...,VERDADEIRO
13,As crianças ficaram quatro meses sem receber o...,IMPRECISO


In [4]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

def get_embs_bertinbau(frase):
    input_ids = tokenizer.encode(frase, return_tensors='pt')
    with torch.no_grad():
        outs = model(input_ids)
        encoded = outs[0][0, 1:-1] 
    return encoded

def get_sentence_embs(frase):
    frase_embs = get_embs_bertinbau(frase.lower()).numpy()
    frase_embs_considerados = []
    palavras = word_tokenize(frase.lower())
    for palavra, embedding in zip(palavras, frase_embs):
            frase_embs_considerados.append(embedding)
    frase_embs_considerados = np.array(frase_embs_considerados)
    final = np.mean(frase_embs_considerados, axis=0)
    return final

dataset['feature'] = dataset['citation'].apply(get_sentence_embs)

In [5]:
dataset.to_csv("processado.csv")

In [6]:
dataset.dropna(inplace=True)

In [7]:
dataset.head()

,citation,category,feature
1,Não se justifica um bilionário aporte financei...,CONTRADITÓRIO,"[0.31005707, -0.1157816, 0.1631995, -0.0117724..."
2,A questão de vacinação ou não é uma lei de 75 ...,EXAGERADO,"[0.014645368, -0.014333419, 0.41869062, 0.2214..."
3,"Inclusive tem um vídeo, do senhor João Doria, ...",VERDADEIRO,"[0.026814118, -0.30709812, 0.4208232, 0.252973..."
4,Hoje em dia pelo menos metade da população diz...,VERDADEIRO,"[-0.056797102, -0.19582988, 0.36105445, -0.084..."
13,As crianças ficaram quatro meses sem receber o...,IMPRECISO,"[-0.14927018, -0.0529135, 0.5366471, -0.187571..."


In [8]:
dataset.shape

(1336, 3)

In [9]:
dataset['category'].value_counts()

VERDADEIRO       464
IMPRECISO        286
FALSO            268
EXAGERADO        169
INSUSTENTÁVEL     96
CONTRADITÓRIO     53
Name: category, dtype: int64

In [10]:
le = preprocessing.LabelEncoder()

In [11]:
X = dataset['feature']
X = [list(a) for a in X]
y = le.fit(dataset['category']).transform(dataset['category'])

In [12]:
le.classes_

array(['CONTRADITÓRIO', 'EXAGERADO', 'FALSO', 'IMPRECISO',
       'INSUSTENTÁVEL', 'VERDADEIRO'], dtype=object)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
sv = svm.SVC()
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

               precision    recall  f1-score   support

CONTRADITÓRIO       0.00      0.00      0.00        11
    EXAGERADO       0.00      0.00      0.00        31
        FALSO       0.27      0.21      0.23        53
    IMPRECISO       0.41      0.11      0.17        66
INSUSTENTÁVEL       0.00      0.00      0.00        24
   VERDADEIRO       0.35      0.88      0.50        83

     accuracy                           0.34       268
    macro avg       0.17      0.20      0.15       268
 weighted avg       0.26      0.34      0.24       268



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
